In [1]:
import torch
import random
from datasets import load_dataset
from sklearn.metrics import accuracy_score
from transformers import AutoModelForCausalLM, AutoTokenizer


In [2]:
if torch.cuda.is_available():
    device = "cuda"

print(device)

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True).to(device)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)


cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
dataset = load_dataset("financial_phrasebank", 'sentences_allagree')
dataset = dataset['train']


In [4]:
random.seed(10)
sampled_indices = random.sample(range(len(dataset)), 1000)
sampled_data = [dataset[i] for i in sampled_indices]

sampled_sentences = [data['sentence'] for data in sampled_data]
sampled_labels = [data['label'] for data in sampled_data]
print(sampled_sentences[5])
print(sampled_labels[5])


Finnish electronics manufacturing services EMS company Elcoteq SE posted a net loss of 66.4 mln euro $ 91.2 mln for the first half of 2007 , compared to a net profit of 7.1 mln euro $ 9.8 mln for the same period of 2006 .
0


In [16]:
prompt = "Instruct: As a Financial News Article Sentiment Classification expert, analyze the sentiment of the following financial news article and classify its sentiment as a single number, 0 being 'Negative', 1 being 'Neutral', and 2 being 'Positive'. Provide only a single number as output, without additional text. Below is an example to illustrate the sentiment classification:\n Example: 'Finnish electronics manufacturing services EMS company Elcoteq SE posted a net loss of 66.4 mln euro $ 91.2 mln for the first half of 2007 , compared to a net profit of 7.1 mln euro $ 9.8 mln for the same period of 2006 .' - Output:0 \n Sentiment Classification news article: "


In [17]:
predictions = []

for sentence in sampled_sentences:
    inputs = tokenizer([prompt + "'" + sentence + "' \n Output:"], return_tensors="pt", return_attention_mask=False).to(device)
    outputs = model.generate(**inputs, max_new_tokens=20, pad_token_id=model.config.eos_token_id)
    text = tokenizer.batch_decode(outputs)
    predictions.append(int(text[-1][-15]))



In [18]:
accuracy = accuracy_score(sampled_labels, predictions)
print(accuracy)

0.675
